# Comparison between normal and log-normal sampling: U-235

In [ ]:
import sandy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
nsmp = 1e5

In [ ]:
cov = sandy.get_endf6_file("jeff_33", "xs", 922350).get_errorr(err=1, errorr33_kws=dict(ign=19), chi=False, mubar=False, nubar=False)["errorr33"].get_cov()
c = cov.data.values

Check 2-norm to see if sample covariance and sample mean match the original covariance and mean

In [ ]:
s_ln = cov.sampling(nsmp, pdf="lognormal")
c_ln = s_ln.get_cov().values
m_ln = s_ln.get_mean().values

print(
    np.linalg.norm(c_ln - c) / np.linalg.norm(c) * 100,
    np.linalg.norm(m_ln - np.ones(m_ln.size)) / np.linalg.norm(np.ones(m_ln.size)) * 100,
)

s_n= cov.sampling(nsmp, pdf="normal")
c_n = s_n.get_cov().values
m_n = s_n.get_mean().values

print(
    np.linalg.norm(c_n - c) / np.linalg.norm(c) * 100,
    np.linalg.norm(m_n - np.ones(m_n.size)) / np.linalg.norm(np.ones(m_ln.size)) * 100,
)

## Plot correlation

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6), dpi=100, sharex=True, sharey=True)

cc = s_n.get_corr()
sns.heatmap(cc, vmin=-1, vmax=1, cmap='bwr', ax=axs[0])

cc = s_ln.get_corr()
sns.heatmap(cc, vmin=-1, vmax=1, cmap='bwr', ax=axs[1])

fig.tight_layout()

## Plot stdev

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4), dpi=100)

s = s_n.get_rstd().reset_index()
s["MT"] = s["MT"].astype("category")
s["ERIGHT"] = [x.right for x in s.E]
s["RSTD"] *= 100
sns.lineplot(data=s, x="ERIGHT", y="RSTD", hue="MT", palette="tab20", ax=ax, drawstyle="steps-pre")

s = s_ln.get_rstd().reset_index()
s["MT"] = s["MT"].astype("category")
s["ERIGHT"] = [x.right for x in s.E]
s["RSTD"] *= 100
sns.lineplot(data=s, x="ERIGHT", y="RSTD", hue="MT", palette="tab20", ax=ax, drawstyle="steps-pre", legend=None, ls="--")

s = cov.get_std().reset_index()
s["MT"] = s["MT"].astype("category")
s["ERIGHT"] = [x.right for x in s.E]
s["STD"] *= 100
sns.lineplot(data=s, x="ERIGHT", y="STD", hue="MT", palette=['k'] * s.MT.unique().size, ax=ax, drawstyle="steps-pre", legend=None, ls="-.", lw=.5)

ax.set(xscale="log")

fig.tight_layout()

## Plot mean

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4), dpi=100)

s = s_n.get_mean().reset_index()
s["MT"] = s["MT"].astype("category")
s["ERIGHT"] = [x.right for x in s.E]
sns.lineplot(data=s, x="ERIGHT", y="MEAN", hue="MT", palette="tab20", ax=ax, drawstyle="steps-pre")

s = s_ln.get_mean().reset_index()
s["MT"] = s["MT"].astype("category")
s["ERIGHT"] = [x.right for x in s.E]
sns.lineplot(data=s, x="ERIGHT", y="MEAN", hue="MT", palette="tab20", ax=ax, drawstyle="steps-pre", legend=None, ls="--")

ax.set(xscale="log")

fig.tight_layout()